# Homework 6: Transformers

The goals of this assignment are:
1. Develop a better understanding of the *self-attention mechanism* in Transformers by implementing it in numpy. 
2. Understand and train a BERT-based model a variant on *coreference resolution.* 
3. Strengthen your understanding of using HuggingFace's `transformers` package. 

## Organization and Instructions
Execute the code cells in Part 1 to understand the background for this assignment. You will not need to modify or add anything to Part 1. Part 2 is where your solution begins.

**Part 1: Background.** 
- 1A. Environment set-up 
- 1B. Data exploration 

**Part 2: Your implementation.** 
- 2A. Self-attention 
- 2B. Zero-shot predictions 
- 2C. Fine-tuning 


**Addtional instructions.** 
- Please follow the 50-foot rule. Your submitted solution and code must be yours alone. Copying and pasting a solution from the internet or another source is considered a violation of the honor code. 

**Evaluation.** Your solution will be evaluated *manually* by the TAs and instructor. 

To help bridge the gap between previous homeworks and the final project. We are **not giving you an autograder**. We hope to help wean you off the grader and give you practice testing your own code.

Please come see us during help hours if you need additional assistance! 

## 1A. Environment Set-up 

If you set-up your conda environment correctly in HW0, you should see `Python [conda env:cs375]` as the kernel in the upper right-hand corner of the Jupyter webpage you are currently on. Run the cell below to make sure your environment is correctly installed. 

In [1]:
# Environment check 
# Return to HW0 if you run into errors in this cell 
# Do not modify this cell 
import os
assert os.environ['CONDA_DEFAULT_ENV'] == "cs375"

import sys
assert sys.version_info.major == 3 and sys.version_info.minor == 11

If there are any errors after running the cell above, return to the instructions from `HW0`. If you are still having difficulty, reach out to the instructor or TAs via Piazza. 

#### Installing other packages

In [2]:
import re
import typing
from typing import List
import numpy as np
import torch
import torch.nn.functional as F
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer, DataCollatorWithPadding)
from datasets import Dataset
from sklearn.metrics import f1_score

In [3]:
import util #inspect util.py to see what is in this file 

## 1B. Data exploration

In [4]:
from datasets import load_dataset

In [5]:
wsc_data = load_dataset("coref-data/davis_wsc_raw", "wsc273", trust_remote_code=True)

In [6]:
wsc_data

DatasetDict({
    test: Dataset({
        features: ['text', 'pronoun', 'pronoun_loc', 'quote', 'quote_loc', 'options', 'label', 'source'],
        num_rows: 273
    })
})

In [7]:
wsc_data['test'][2]

{'text': "The trophy doesn't fit into the brown suitcase because it is too large.",
 'pronoun': 'it',
 'pronoun_loc': 55,
 'quote': 'it is too large',
 'quote_loc': 55,
 'options': ['the trophy', 'the suitcase'],
 'label': 0,
 'source': 'Hector Levesque'}

In [8]:
wsc_data['test'][3]

{'text': "The trophy doesn't fit into the brown suitcase because it is too small.",
 'pronoun': 'it',
 'pronoun_loc': 55,
 'quote': 'it is too small',
 'quote_loc': 55,
 'options': ['the trophy', 'the suitcase'],
 'label': 1,
 'source': 'Hector Levesque'}

In this homework, we will use the WinoGrande dataset. You can read more about the dataset in [this paper](https://cdn.aaai.org/ojs/6399/6399-13-9624-1-10-20200517.pdf). 

Here is Table 1 from the WinoGrande paper with examples:  

![](figs/winograd.png)

HuggingFace provides a Python package for loading (and uploading datasets). You can read more about the `datasets` Python package [here](https://huggingface.co/docs/datasets/en/index). 

In [9]:
from datasets import load_dataset

In [10]:
# Load the WinoGrande dataset
dataset = load_dataset("allenai/winogrande", "winogrande_s", trust_remote_code=True)

# Access the training and validation splits
train_dataset = dataset["train"]
validation_dataset = dataset["validation"].select(range(500)) #We'll just look at 200 dev exs

print(f"Num. train exs= {len(train_dataset)}")
print(f"Num. dev exs= {len(validation_dataset)}")

Num. train exs= 640
Num. dev exs= 500


In [11]:
# Let's look at one example from the validation dataset
print(validation_dataset[12])

{'sentence': 'I had to read an entire story for class tomorrow. Luckily, the _ was short.', 'option1': 'story', 'option2': 'class', 'answer': '1'}


Above, the `'sentence'` is the full sentence with a `_` for where the pronoun or noun options should go. 

Then `option1` and `option2` are the two token spans from the sentence the model will eventually choose from and `answer` is the correct answer. 

## 2A. Self-attention

In this part, you will implement the parallelized version of the *masked* self-attention mechanism in Transformers using only numpy.


Recall, for each layer $k$ in the transformer block we have 

For a single example with $n$ tokens and embedding dimension $d$, we first have $X^k$, the contextual embedding matrix (size $n\times d$) for layer $k$. 

Then, we introduce the weights, 

$$ Q = X^k \times W_Q$$ 
$$ K = X^k \times W_K$$
$$ V = X^k \times W_V $$

and use the new matrices to get the contextual embedding matrix for the next layer, 

$$ X^{k+1} = \text{softmax} \bigg( \text{mask} \bigg( \frac{QK^T}{\sqrt{d}} \bigg) \bigg) V$$

This is computationally efficient in a matrix-multiplication-optimized library like `numpy` because it should have **no for-loops!** 

Let's implement self-attention for the (modified) example we were looking at in Part 1 

*"I had to read an entire story for class tomorrow. Luckily, it was short."*

In [12]:
# Tokens for our example 
toks = ["i", "had", "to", "read", "an", 
        "entire", "story", "for", "class", "tomorrow", ".",
       "luckily", "it", "was", "short", "."]

In [13]:
# Load pre-specified embeddings and weights (for testing)
X, W_Q, W_K, W_V = util.load_attention_data(toks)

In [14]:
# TODO: Implement your approach in this function

def self_attention(X: np.ndarray, W_Q: np.ndarray, 
                   W_K: np.ndarray, W_V: np.ndarray) -> np.ndarray: 
    """
    Implements (masked) self-attention mechanism for a single layer 
    (and a single example)
    
    Returns: X_new, a np.ndarray that is the same shape as X
    
    Notes:
    - You can only use numpy for this part of the homework and no other packages
    - Your solution must not have any for-loops!
    
    Tips: 
        - Double-check the shapes of all the matrices you're working with. 
        - We recommend making a helper function for the softmax.
        - You may a subset of these numpy methods and operators helpful: `np.exp`, `@`, `np.triu_indicies`, `np.reshape`, `np.inf`, `np.broadcast_to`, `np.choose`.  
    """
    pass 

## 2B. Zero-shot predictions

Now, we will use a distilled version of "RoBERTa" (a BERT variant) to make zero-shot predictions on the WinoGrande dataset. 

#### Tokenization and pre-processing

In [15]:
model_name = "distilbert/distilbert-base-uncased"

In [16]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=True)

In [17]:
wsc_data['test'][0]

{'text': 'The city councilmen refused the demonstrators a permit because they feared violence.',
 'pronoun': 'they',
 'pronoun_loc': 63,
 'quote': 'they feared violence',
 'quote_loc': 63,
 'options': ['The city councilmen', 'The demonstrators'],
 'label': 0,
 'source': '(Winograd 1972)'}

In [18]:
# First example in the dev dataset (see dataset loading in Part 1B above)
text1 = wsc_data['test'][0]['text']
text1

'The city councilmen refused the demonstrators a permit because they feared violence.'

In [19]:
text1.find("the demonstrators")

28

In [20]:
text1 = validation_dataset[0]['sentence']
text1

'Sarah was a much better surgeon than Maria so _ always got the easier cases.'

In [21]:
# Converts to tokens and attention mask 
# The attention mask will be 0 if there are special "PAD" tokens
inputs = tokenizer(text1, return_tensors="pt", return_offsets_mapping=True)
inputs

{'input_ids': tensor([[ 101, 4532, 2001, 1037, 2172, 2488, 9431, 2084, 3814, 2061, 1035, 2467,
         2288, 1996, 6082, 3572, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'offset_mapping': tensor([[[ 0,  0],
         [ 0,  5],
         [ 6,  9],
         [10, 11],
         [12, 16],
         [17, 23],
         [24, 31],
         [32, 36],
         [37, 42],
         [43, 45],
         [46, 47],
         [48, 54],
         [55, 58],
         [59, 62],
         [63, 69],
         [70, 75],
         [75, 76],
         [ 0,  0]]])}

In [22]:
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
tokens

['[CLS]',
 'sarah',
 'was',
 'a',
 'much',
 'better',
 'surgeon',
 'than',
 'maria',
 'so',
 '_',
 'always',
 'got',
 'the',
 'easier',
 'cases',
 '.',
 '[SEP]']

Note: Above, when we see the `###` characters before tokens, this means the BERT tokenizer has split a word into subwords.

In [23]:
# TODO: fill in the function below 
def which_tok_index(tok_string: str, sentence: str, inputs: torch.tensor) -> int:
    """
    Inputs: the token string (tok_string) of interest, the original sentence
    and the tokenized input tensors 
    
    Returns: (int) the first index that matches the tok_string. 
    
        If tok_string gets tokenized into multiple tokens, return the *first* index corresponding
        to the multi-token span. 

        If there is no match (which could happen), return 0. 
    
    Example: 
        tok_string="sarah"
        
        input_ids = {'input_ids': tensor([[ 101, 4532, 2001, 1037, 2172, 2488, 9431, 2084, 3814, 2061, 1035, 2467,
         2288, 1996, 6082, 3572, 1012,  102]]), 
                     'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 
                     'offset_mapping': tensor([[[ 0,  0],
                     [ 0,  5],
                     [ 6,  9],
                     [10, 11],
                     [12, 16],
                     [17, 23],
                     [24, 31],
                     [32, 36],
                     [37, 42],
                     [43, 45],
                     [46, 47],
                     [48, 54],
                     [55, 58],
                     [59, 62],
                     [63, 69],
                     [70, 75],
                     [75, 76],
                     [ 0,  0]]])}
        
        Returns: 1 
        
        This example returns 1 since the token id at index 1 (value 432)
        starts at character index 6 in the orginal sentence and corresponds to "sarah."
        
    Tips:  
        - Understaning 'offset_mapping' and Python string methods may be helpful here
        - tokenizer.convert_ids_to_tokens could help with debugging 
    """
    # find the start character
    start_indx_goal = sentence.find(tok_string)
    
    for i, tok in enumerate(inputs['offset_mapping'][0]):
        start, end = tok
        start = start.item()
        end = end.item()
        if end == 0: continue 
        #see if we find a match 
        if start_indx_goal == start: return i 
    
    return 0 # delete and replace with your code 

In [24]:
# Unit test
sent = validation_dataset[0]['sentence']
inputs = tokenizer(sent, return_tensors="pt", return_offsets_mapping=True)

t1 = which_tok_index("sarah", sent.lower(), inputs)
print(t1, "== 1?")
t2 = which_tok_index("maria", sent.lower(), inputs)
print(t2, "== 8?")
t3 = which_tok_index("_", sent.lower(), inputs)
print(t3, "== 10?")

1 == 1?
8 == 8?
10 == 10?


In [25]:
# Another unit test 
sent3 = validation_dataset[3]['sentence']
inputs3 = tokenizer(sent3, return_tensors="pt", return_offsets_mapping=True)

t1 = which_tok_index(validation_dataset[3]['option1'].lower(), sent3, inputs3)
print(t1, "== 7?")
t2 = which_tok_index(validation_dataset[3]['option2'].lower(), sent3, inputs3)
print(t2, "== 12?")
t3 = which_tok_index("blah", sent3, inputs3)
print(t3, "== 0?")

7 == 7?
12 == 12?
0 == 0?


#### Zero-shot prediction

Now, we'll use the model to make zero-shot predictions. Note, this is "zero-shot" because we haven't ever trained the model on this particular task or dataset.  

Here's how we will make zero-shot predictions: 
1. Pass the sentence (after tokenization) into the pre-trained model 
2. Obtain the final layer contextual embeddings for the `"_"` token as well as the *first* token for the substring of `option1` and likewise for `option2`. 
3. Find the cosine similarity between these contextual embeddings between `"_"` and the embedding we chose for `option1` and likewise for `option2`. 
4. Whichever has the higher cosine similarity (`option1` or `option2`), make this the prediction. 

In [26]:
# TODO: Your implementation
def zero_shot_predictions(model, tokenizer, dataset) -> List[int]: 
    """
    Make zero-shot predictions with the last layer contextual embedding
    cosine similarity method described in the previous cell. 
    
    Returns: 
        List[str], a list of strings, one element for each 
        example in the input dataset. Each element is an int: 
            - 1 corresponding to "option1" in the dataset
            - 2 corresponding to "option2" in the dataset
    
    Note: 
        - For now, it's ok if you have a for-loop over examples. 
          (In an actual industry setting, you would make this all parallelized)
        - You might make use of the `which_tok_index()` helper function 
        you just implemented 
        - The documentation on AutoModelForSequenceClassification may be helpful here. 
        - torch.nn.functional may have some helpful methods 
        - Using model.eval() and torch.no_grad() will speed things up (since Pytorch will not
            have to make the computation graph)
    """
    model.eval()
    preds = []
    
    for i in range(len(dataset)):
        # Get the contextual embeddings (last hidden states)
        text = dataset[i]['sentence']
        inputs = tokenizer(text, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
        last_hidden_states = outputs.hidden_states[-1][0]

        #Now get the embedding for "_"
        inputs = tokenizer(text, return_tensors="pt", return_offsets_mapping=True)
        underscore_indx = which_tok_index("_", text.lower(), inputs)
        underscore_embed = last_hidden_states[underscore_indx]

        # Embedding for option1
        option1 = dataset[i]['option1'].lower()
        option1_idx =  which_tok_index(option1, text.lower(), inputs)
        option1_embed = last_hidden_states[option1_idx]

        #Embedding for option2
        option2 = dataset[i]['option2'].lower()
        option2_idx =  which_tok_index(option2, text.lower(), inputs)
        option2_embed = last_hidden_states[option2_idx]
        
        #print(i, "indexes=", underscore_indx, option1_idx, option2_idx)

        #Now cosine sims 
        #print(option1, option2)
        cos_sim1 = F.cosine_similarity(underscore_embed, option1_embed, dim=0)
        cos_sim2 = F.cosine_similarity(underscore_embed, option2_embed, dim=0)

        #Predictions
        try: 
            if cos_sim1 > cos_sim2: 
                preds.append(1)
            else: 
                preds.append(2)
        except: 
            print(i, cos_sim1, cos_sim2)

    return preds

In [27]:
print(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
# You might see a warning below. 
# We'll end up doing this in the next part of the homework;)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


distilbert/distilbert-base-uncased


In [28]:
# Test your code on just a single example 
zero_shot_predictions(model, tokenizer, validation_dataset.select(range(1)))

[2]

In [29]:
# Make the full predictions 
preds = zero_shot_predictions(model, tokenizer, validation_dataset)

In [30]:
# Uncomment and check the following
#len(preds) == len(validation_dataset)

#### Evaluation

In [31]:
try: 
    truth = [int(x['answer']) for x in validation_dataset]
    assert len(truth) == len(preds)
    y_true = np.array(truth)
    y_pred = np.array(preds)
    y_baseline = np.ones(len(y_true)) *2
    print("F1 of baseline (maj. class)=", np.round(f1_score(y_true, y_baseline, pos_label=2), 2))
    print("F1 of zero-shot=", np.round(f1_score(y_true, y_pred, pos_label=2), 2))
except: 
    print("Need len(truth) == len(preds)")

F1 of baseline (maj. class)= 0.67
F1 of zero-shot= 0.5


How does your zero-shot model compare to the baseline? 

##  2C. Fine-tuning

Now we'll fine-tune our model on the training dataset. We'll give you some code to help with the pre-processing. It's your job to use `TrainingArguments` and `Trainer` from HuggingFace in order to train the model. 

Tips: 
- [This](https://huggingface.co/docs/transformers/en/tasks/sequence_classification) HuggingFace tutorial may be helpful. 

In [32]:
model_name =  "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
# KATIE TODO: replace the "_"! Duh! 

In [ ]:
def substitute_winograde_data(dataset):
    """
    In this preprocessing function for classification, we create two training 
    examples per original example by substituting option1 or option2 for _. 
    
    For instance, with the example 
        {'sentence': 'I had to read an entire story for class tomorrow. 
                        Luckily, the _ was short.', 
        'option1': 'story', 'option2': 'class', 'answer': '1'}
        
    We create 
        x1 = "I had to read an entire story for class tomorrow. Luckily, the story was short."
        y1= 1 (correct)
        
        x2 = "I had to read an entire story for class tomorrow. Luckily, the class was short."
        y2 = 0 (incorrect)
    """
    
    out = []
    for i, x in enumerate(dataset):
        new_x = {}
        # Create a "positive answer"
        if x['answer'] == '1': 
            replace_str = x['option1']
        else: 
            replace_str = x['option2']
        new_x['text'] = x['sentence'].replace("_", replace_str)
        new_x['label'] = 1 # y=1
        out.append(new_x)

        #Then a "negative answer"
        if x['answer'] == '1': 
            replace_str = x['option2'] # Opposite!!! 
        else: 
            replace_str = x['option1']
        new_x['text'] = x['sentence'].replace("_", replace_str)
        new_x['label'] = 0 # y=0
        out.append(new_x)
    return out

In [35]:
train_for_classification = Dataset.from_list(substitute_winograde_data(train_dataset))
train_for_classification

Dataset({
    features: ['text', 'label'],
    num_rows: 1280
})

In [36]:
dev_for_classification = Dataset.from_list(substitute_winograde_data(validation_dataset))
dev_for_classification

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

In [37]:
def preprocess_function(dataset):
    return tokenizer(dataset["text"], truncation=True, padding="max_length", max_length=100)

# def add_labels(example, idx):
#     example['label'] = int(example['answer'])-1
#     return example

# encoded_train = train_dataset.map(preprocess_function, batched=True)
# encoded_train = encoded_train.map(add_labels, with_indices=True)


In [38]:
encoded_train = train_for_classification.map(preprocess_function, batched=True)

Map:   0%|          | 0/1280 [00:00<?, ? examples/s]

In [39]:
encoded_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1280
})

In [40]:
encoded_dev = dev_for_classification.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

TODO: It's your job to use `TrainingArguments` and `Trainer` from HuggingFace in order to train `model_tune` above! 

In [41]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [42]:
import evaluate
f1_score_func = evaluate.load("f1")

In [43]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1_score_func.compute(predictions=predictions, references=labels)

In [44]:
training_args = TrainingArguments(
    output_dir="saved",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False, # NO HUB NEEDED! 
)

In [45]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_dev,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Be sure to report your final F1 score on the validation dataset. 

In [ ]:
## TODO: put your eval code here ### 

In [ ]:
predictions_output = trainer.predict(encoded_dev)

In [ ]:
metrics = predictions_output.metrics
print(metrics)

In [ ]:
def preprocess_function2(dataset):
    return tokenizer(dataset["text"], truncation=True, padding="max_length", max_length=100)

In [ ]:
# Predict on the WSC data 
wsc_encode = wsc_data.map(preprocess_function2, batched=True)

In [ ]:
predictions_output = trainer.predict(wsc_encode['test'])
metrics = predictions_output.metrics
print(metrics)

## Submission

In [ ]:
%%bash

if [[ ! -f "./hw6.ipynb" ]]
then
    echo "WARNING: Did not find notebook in Jupyter working directory. Manual solution: go to File->Download .ipynb to download your notebok and other files, then zip them locally."
else
    echo "Found notebook file, creating submission zip..."
    zip -r submission.zip hw6.ipynb
fi